In [47]:
import numpy as np
from scipy.linalg import expm
import random

class move_gym():
    def __init__(self):
        self.scope = 2.0
        self.states = 6
        self.actions = 4
    def reset(self, s=[]):
        if np.array( s ).shape[0] == 0:
            self.obstacle_x = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
            self.obstacle_y = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1

            self.move_x = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
            self.move_y = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1  
            while ( abs(self.obstacle_x-self.move_x) < 1.05 ) & ( abs(self.obstacle_y-self.move_y) < 1.05 ):
                self.move_x = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
                self.move_y = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1      

            self.target_x = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
            self.target_y = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
            while (  ( abs(self.obstacle_x-self.target_x) < 1.05 ) & ( abs(self.obstacle_y-self.target_y) < 1.05 )  ) |\
             (  ( abs(self.move_x-self.target_x) < 1.05 ) & ( abs(self.move_y-self.target_y) < 1.05 )  ):
                self.target_x = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1
                self.target_y = np.random.randint( int(-self.scope*10), int(self.scope*10) )*0.1        
        else:
            self.obstacle_x=s[0]
            self.obstacle_y=s[1]
            self.target_x=s[2]
            self.target_y=s[3]
            self.move_x=s[4]
            self.move_y=s[5]

        state=np.array([ self.obstacle_x, self.obstacle_y, self.target_x, self.target_y, self.move_x, self.move_y ])
        return state #, self.state2img(state)
    def step(self, action, collision_limit = 1.05):
        velocity = 0.2
        if action==0: # up down right left
            self.move_y+=velocity
        if action==1:
            self.move_y-=velocity
        if action==2:
            self.move_x+=velocity
        if action==3:
            self.move_x-=velocity

#         if self.move_x > (self.scope+1.0):
#             self.move_x-=velocity
#         if self.move_x < (-self.scope-1.0):
#             self.move_x+=velocity
#         if self.move_y > (self.scope+1.0):
#             self.move_y-=velocity
#         if self.move_y < (-self.scope-1.0):
#             self.move_y+=velocity
        
        reward = -0.1
        done = False
        info = "^_^"
        if (  ( abs(self.obstacle_x-self.move_x) < collision_limit ) & ( abs(self.obstacle_y-self.move_y) < collision_limit )  ): 
            reward = -1.0 
            done = True
            info = "collision"

        elif (  ( abs(self.target_x-self.move_x) < 1.05 ) & ( abs(self.target_y-self.move_y) < 1.05 )  ):
            reward = 1.0
            done = True
            info = "reach"

        state=np.array([ self.obstacle_x, self.obstacle_y, self.target_x, self.target_y, self.move_x, self.move_y ])

        return state, reward,done,info
    def state2img(self, state):
        img = np.zeros([84,84,3], dtype=np.uint8)
        ( img[ int(42-(self.obstacle_y+0.5)*10):int(42-(self.obstacle_y-0.5)*10), int((self.obstacle_x-0.5)*10+42):int((self.obstacle_x+0.5)*10+42),2 ] ).fill(255)
        ( img[ int(42-(self.target_y+0.5)*10):int(42-(self.target_y-0.5)*10), int((self.target_x-0.5)*10+42):int((self.target_x+0.5)*10+42),1 ] ).fill(255)
        ( img[ int(42-(self.move_y+0.5)*10):int(42-(self.move_y-0.5)*10), int((self.move_x-0.5)*10+42):int((self.move_x+0.5)*10+42),0 ] ).fill(255)
        return img

In [48]:
env = move_gym()
STEP = 50

In [2]:
def discrete_it(state, discrete):
    
    dis_s = []
    for i in xrange( len(state) ):
        num = state[i]
        over = num - discrete
        for j in xrange( len(over) ):
            if over[j]<0:
                over[j] = 100 # a certain big number
        dis_s.append( np.argmin( over ) )
    return dis_s

In [3]:
def state2relative(state):
    top_t = 100.0
    bottom_t = 100.0
    right_t = 100.0
    left_t = 100.0

    top_o = 100.0
    bottom_o = 100.0
    right_o = 100.0
    left_o = 100.0        

    obstacle_x=state[0]
    obstacle_y=state[1]
    target_x=state[2]
    target_y=state[3]
    move_x=state[4]
    move_y=state[5]

    if target_y >= move_y:
        top_t = target_y - move_y
    if target_y < move_y:
        bottom_t = move_y - target_y
    if target_x >= move_x:
        right_t = target_x - move_x
    if target_x < move_x:
        left_t = move_x - target_x

    if obstacle_y >= move_y:
        top_o = obstacle_y - move_y
    if obstacle_y < move_y:
        bottom_o = move_y - obstacle_y
    if obstacle_x >= move_x:
        right_o = obstacle_x - move_x
    if obstacle_x < move_x:
        left_o = move_x - obstacle_x

    return np.array([ top_t, bottom_t, right_t, left_t, top_o, bottom_o, right_o, left_o ])

In [220]:
logic_q = np.load("logic_q-1.02-4-adjust-9.npy")

In [10]:
discrete = np.array( [0., 1.02, 1.86, 2.93, 4.1, 5.03] ) 

In [202]:
from tqdm import trange

In [224]:
B = []
B1 = []
for i in trange( 1000000 ):
    state = env.reset(  )
    A = state
    old_action = 1.5
    for j in xrange(STEP):

        relative_state = state2relative(state)
        # discrete it:
        relative_state = discrete_it( relative_state, discrete )

        action = np.argmax( logic_q[relative_state[0]][relative_state[1]][relative_state[2]][relative_state[3]]\
        [relative_state[4]][relative_state[5]][relative_state[6]][relative_state[7]] ) - 1
        
        
        if (((relative_state[0]+relative_state[4])==0) | ((relative_state[1]+relative_state[5])==0)) & (action<1.2) & (old_action<1.2):
#             print "^^^"
            action = old_action
        if (((relative_state[2]+relative_state[6])==0) | ((relative_state[3]+relative_state[7])==0)) & (action>1.8) & (old_action>1.8):
#             print "%%%"
            action = old_action
        
        old_action = action
        state, reward,done,info = env.step(action, 1.05)
        if done:
            if info=="collision":
                B1.append(A)
            break
    if done == 0:
        B.append(A)

print "len(B): ", len(B), "len(B1): ", len(B1)

100%|██████████| 1000000/1000000 [06:14<00:00, 2673.46it/s]

len(B):  0 len(B1):  0


In [218]:
D=B
D

[array([ 1.3, -2. , -2. ,  1.9, -0.5, -0.1]),
 array([ 1.5,  1.9,  0.6, -2. , -1.3, -0.1]),
 array([-0.1, -0.3, -2. ,  1.9,  1.4, -2. ]),
 array([ 1.5,  1.8,  1.8, -2. , -0.7, -0.1]),
 array([ 0.2, -0.9, -1.7,  1.8,  1.9, -2. ]),
 array([ 0.4, -0.4, -1.5,  1.7,  1.8, -2. ])]

In [222]:
for i in xrange( len(D) ):
    print i, "------------------------------"
    state = env.reset( D[i] )
    A = state
    old_action = 1.5
    for j in xrange( STEP ):
        print j,'>>'
        
        relative_state = state2relative(state)
        # discrete it:
        relative_state = discrete_it( relative_state, discrete )
        
        print state, relative_state,
        
        action = np.argmax( logic_q[relative_state[0]][relative_state[1]][relative_state[2]][relative_state[3]]\
        [relative_state[4]][relative_state[5]][relative_state[6]][relative_state[7]] ) - 1
        
        if (((relative_state[0]+relative_state[4])==0) | ((relative_state[1]+relative_state[5])==0)) & (action<1.2) & (old_action<1.2):
#             print "^^^"
            action = old_action
        if (((relative_state[2]+relative_state[6])==0) | ((relative_state[3]+relative_state[7])==0)) & (action>1.8) & (old_action>1.8):
#             print "%%%"
            action = old_action
            
        state, reward,done,info = env.step(action, 1.05)
        old_action = action
        
        print action, state
        
        if done:
            print "info: ", info
            break
    if done == 0:
        pass

0 ------------------------------
0 >>
[ 1.3 -2.  -2.   1.9 -0.5 -0.1] [2, 5, 5, 1, 5, 2, 1, 5] 3 [ 1.3 -2.  -2.   1.9 -0.7 -0.1]
1 >>
[ 1.3 -2.  -2.   1.9 -0.7 -0.1] [2, 5, 5, 1, 5, 2, 2, 5] 0 [ 1.3 -2.  -2.   1.9 -0.7  0.1]
2 >>
[ 1.3 -2.  -2.   1.9 -0.7  0.1] [1, 5, 5, 1, 5, 2, 2, 5] 3 [ 1.3 -2.  -2.   1.9 -0.9  0.1]
3 >>
[ 1.3 -2.  -2.   1.9 -0.9  0.1] [1, 5, 5, 1, 5, 2, 2, 5] 3 [ 1.3 -2.  -2.   1.9 -1.1  0.1]
4 >>
[ 1.3 -2.  -2.   1.9 -1.1  0.1] [1, 5, 5, 0, 5, 2, 2, 5] 0 [ 1.3 -2.  -2.   1.9 -1.1  0.3]
5 >>
[ 1.3 -2.  -2.   1.9 -1.1  0.3] [1, 5, 5, 0, 5, 2, 2, 5] 0 [ 1.3 -2.  -2.   1.9 -1.1  0.5]
6 >>
[ 1.3 -2.  -2.   1.9 -1.1  0.5] [1, 5, 5, 0, 5, 2, 2, 5] 0 [ 1.3 -2.  -2.   1.9 -1.1  0.7]
7 >>
[ 1.3 -2.  -2.   1.9 -1.1  0.7] [1, 5, 5, 0, 5, 2, 2, 5] 0 [ 1.3 -2.  -2.   1.9 -1.1  0.9]
info:  reach
1 ------------------------------
0 >>
[ 1.5  1.9  0.6 -2.  -1.3 -0.1] [5, 2, 2, 5, 2, 5, 2, 5] 1 [ 1.5  1.9  0.6 -2.  -1.3 -0.3]
1 >>
[ 1.5  1.9  0.6 -2.  -1.3 -0.3] [5, 1, 2, 5, 2, 5, 2